# Competição Externa - BigData

Notebook voltado para integrar os dados das lojas atuais com os do IBGE.

## Bibliotecas que serão usadas

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import date

from matplotlib.cm import get_cmap

from pandas_profiling import ProfileReport

from sklearn import tree
from sklearn import preprocessing
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

plt.style.use("ggplot")

## Lendo os dados

In [22]:
data_la = pd.read_csv("data/lojas_atuais.csv")

data_la

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,...,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18
0,1,4125506005002,4125506,NaN,FEATURE_02_VALUE_04,True,21.76,FEATURE_05_VALUE_04,31,0.168922,...,115.958848,2.349821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,5300108005004,5300108,NaN,FEATURE_02_VALUE_03,True,22.00,FEATURE_05_VALUE_03,27,0.215967,...,1562.674817,3.670883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3550308005052,3550308,NaN,FEATURE_02_VALUE_03,True,45.66,FEATURE_05_VALUE_03,6,0.190102,...,5593.905463,2.815883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,3145604003002,3145604,NaN,FEATURE_02_VALUE_04,False,30.00,FEATURE_05_VALUE_03,3,0.141988,...,12.824022,7.218132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1600303005010,1600303,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,False,62.49,FEATURE_05_VALUE_03,30,0.107619,...,325.531120,1.972944,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.0,88.1,100.0,100.0,88.5,93.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3125,3682,5105259003002,5105259,NaN,FEATURE_02_VALUE_04,False,30.16,FEATURE_05_VALUE_03,5,0.210289,...,36.397933,3.491315,NaN,NaN,100.0,91.7,100.0,100.0,97.1,97.6
3126,3683,2604106003003,2604106,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,False,38.00,FEATURE_05_VALUE_03,13,0.146264,...,163.567629,2.016780,NaN,NaN,100.0,85.0,100.0,100.0,50.7,72.5
3127,3685,4115804003002,4115804,NaN,FEATURE_02_VALUE_04,False,44.04,FEATURE_05_VALUE_02,7,0.226926,...,16.862438,4.455832,NaN,NaN,100.0,71.5,100.0,100.0,99.0,95.2
3128,3686,3509700003001,3509700,NaN,FEATURE_02_VALUE_06,False,31.76,FEATURE_05_VALUE_03,4,0.134354,...,20.565405,3.932982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
data_cod_mun_ibge = pd.read_excel("data/RELATORIO_DTB_BRASIL_MUNICIPIO.xls")

data_cod_mun_ibge

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município
0,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste
1,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis
2,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso
3,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste
4,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes
...,...,...,...,...,...,...,...,...,...
5565,52,Goiás,5,Sul Goiano,16,Pires do Rio,22005,5222005,Vianópolis
5566,52,Goiás,5,Sul Goiano,15,Meia Ponte,22054,5222054,Vicentinópolis
5567,52,Goiás,4,Leste Goiano,12,Entorno de Brasília,22203,5222203,Vila Boa
5568,52,Goiás,4,Leste Goiano,12,Entorno de Brasília,22302,5222302,Vila Propício


In [24]:
data_cod_mun_ibge.columns

Index(['UF', 'Nome_UF', 'Mesorregião Geográfica', 'Nome_Mesorregião',
       'Microrregião Geográfica', 'Nome_Microrregião', 'Município',
       'Código Município Completo', 'Nome_Município'],
      dtype='object')

In [25]:
data_cod_mun_ibge.loc[data_cod_mun_ibge["Código Município Completo"] == 4125506].iloc[0].Nome_UF

'Paraná'

In [26]:
data_cod_mun_ibge = data_cod_mun_ibge[["Nome_UF", "Código Município Completo", "Nome_Município"]]

data_cod_mun_ibge

,Nome_UF,Código Município Completo,Nome_Município
0,Rondônia,1100015,Alta Floresta D'Oeste
1,Rondônia,1100379,Alto Alegre dos Parecis
2,Rondônia,1100403,Alto Paraíso
3,Rondônia,1100346,Alvorada D'Oeste
4,Rondônia,1100023,Ariquemes
...,...,...,...
5565,Goiás,5222005,Vianópolis
5566,Goiás,5222054,Vicentinópolis
5567,Goiás,5222203,Vila Boa
5568,Goiás,5222302,Vila Propício


In [27]:
data_cod_mun_ibge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Nome_UF                    5570 non-null   object
 1   Código Município Completo  5570 non-null   int64 
 2   Nome_Município             5570 non-null   object
dtypes: int64(1), object(2)
memory usage: 130.7+ KB


### Faz label enconder nos dados do ibge

In [28]:
le = preprocessing.LabelEncoder()

le.fit(data_cod_mun_ibge["Nome_UF"])

LabelEncoder()

In [29]:
list(le.classes_)

['Acre',
 'Alagoas',
 'Amapá',
 'Amazonas',
 'Bahia',
 'Ceará',
 'Distrito Federal',
 'Espírito Santo',
 'Goiás',
 'Maranhão',
 'Mato Grosso',
 'Mato Grosso do Sul',
 'Minas Gerais',
 'Paraná',
 'Paraíba',
 'Pará',
 'Pernambuco',
 'Piauí',
 'Rio Grande do Norte',
 'Rio Grande do Sul',
 'Rio de Janeiro',
 'Rondônia',
 'Roraima',
 'Santa Catarina',
 'Sergipe',
 'São Paulo',
 'Tocantins']

In [30]:
data_cod_mun_ibge["UF_COD"] = le.transform(data_cod_mun_ibge["Nome_UF"])

data_cod_mun_ibge

,Nome_UF,Código Município Completo,Nome_Município,UF_COD
0,Rondônia,1100015,Alta Floresta D'Oeste,21
1,Rondônia,1100379,Alto Alegre dos Parecis,21
2,Rondônia,1100403,Alto Paraíso,21
3,Rondônia,1100346,Alvorada D'Oeste,21
4,Rondônia,1100023,Ariquemes,21
...,...,...,...,...
5565,Goiás,5222005,Vianópolis,8
5566,Goiás,5222054,Vicentinópolis,8
5567,Goiás,5222203,Vila Boa,8
5568,Goiás,5222302,Vila Propício,8


In [33]:
len(data_cod_mun_ibge["UF_COD"].unique())

27

In [39]:
data_cod_mun_ibge.to_csv("data/municipios-uf.csv", index = False)

In [34]:
data_merge = pd.merge(data_la, data_cod_mun_ibge, left_on = "cod_municipio", right_on = "Código Município Completo", how = "left")

data_merge

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,...,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,Nome_UF,Código Município Completo,Nome_Município,UF_COD
0,1,4125506005002,4125506,NaN,FEATURE_02_VALUE_04,True,21.76,FEATURE_05_VALUE_04,31,0.168922,...,NaN,NaN,NaN,NaN,NaN,NaN,Paraná,4125506.0,São José dos Pinhais,13.0
1,2,5300108005004,5300108,NaN,FEATURE_02_VALUE_03,True,22.00,FEATURE_05_VALUE_03,27,0.215967,...,NaN,NaN,NaN,NaN,NaN,NaN,Distrito Federal,5300108.0,Brasília,6.0
2,3,3550308005052,3550308,NaN,FEATURE_02_VALUE_03,True,45.66,FEATURE_05_VALUE_03,6,0.190102,...,NaN,NaN,NaN,NaN,NaN,NaN,São Paulo,3550308.0,São Paulo,25.0
3,4,3145604003002,3145604,NaN,FEATURE_02_VALUE_04,False,30.00,FEATURE_05_VALUE_03,3,0.141988,...,NaN,NaN,NaN,NaN,NaN,NaN,Minas Gerais,3145604.0,Oliveira,12.0
4,5,1600303005010,1600303,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,False,62.49,FEATURE_05_VALUE_03,30,0.107619,...,100.0,88.1,100.0,100.0,88.5,93.8,Amapá,1600303.0,Macapá,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3125,3682,5105259003002,5105259,NaN,FEATURE_02_VALUE_04,False,30.16,FEATURE_05_VALUE_03,5,0.210289,...,100.0,91.7,100.0,100.0,97.1,97.6,Mato Grosso,5105259.0,Lucas do Rio Verde,10.0
3126,3683,2604106003003,2604106,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,False,38.00,FEATURE_05_VALUE_03,13,0.146264,...,100.0,85.0,100.0,100.0,50.7,72.5,Pernambuco,2604106.0,Caruaru,16.0
3127,3685,4115804003002,4115804,NaN,FEATURE_02_VALUE_04,False,44.04,FEATURE_05_VALUE_02,7,0.226926,...,100.0,71.5,100.0,100.0,99.0,95.2,Paraná,4115804.0,Medianeira,13.0
3128,3686,3509700003001,3509700,NaN,FEATURE_02_VALUE_06,False,31.76,FEATURE_05_VALUE_03,4,0.134354,...,NaN,NaN,NaN,NaN,NaN,NaN,São Paulo,3509700.0,Campos do Jordão,25.0


In [35]:
data_merge = data_merge.drop("Código Município Completo", axis = 1)

data_merge

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,...,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,Nome_UF,Nome_Município,UF_COD
0,1,4125506005002,4125506,NaN,FEATURE_02_VALUE_04,True,21.76,FEATURE_05_VALUE_04,31,0.168922,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paraná,São José dos Pinhais,13.0
1,2,5300108005004,5300108,NaN,FEATURE_02_VALUE_03,True,22.00,FEATURE_05_VALUE_03,27,0.215967,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Distrito Federal,Brasília,6.0
2,3,3550308005052,3550308,NaN,FEATURE_02_VALUE_03,True,45.66,FEATURE_05_VALUE_03,6,0.190102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,São Paulo,São Paulo,25.0
3,4,3145604003002,3145604,NaN,FEATURE_02_VALUE_04,False,30.00,FEATURE_05_VALUE_03,3,0.141988,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Minas Gerais,Oliveira,12.0
4,5,1600303005010,1600303,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,False,62.49,FEATURE_05_VALUE_03,30,0.107619,...,FEATURE_12_VALUE_01,100.0,88.1,100.0,100.0,88.5,93.8,Amapá,Macapá,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3125,3682,5105259003002,5105259,NaN,FEATURE_02_VALUE_04,False,30.16,FEATURE_05_VALUE_03,5,0.210289,...,NaN,100.0,91.7,100.0,100.0,97.1,97.6,Mato Grosso,Lucas do Rio Verde,10.0
3126,3683,2604106003003,2604106,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,False,38.00,FEATURE_05_VALUE_03,13,0.146264,...,NaN,100.0,85.0,100.0,100.0,50.7,72.5,Pernambuco,Caruaru,16.0
3127,3685,4115804003002,4115804,NaN,FEATURE_02_VALUE_04,False,44.04,FEATURE_05_VALUE_02,7,0.226926,...,NaN,100.0,71.5,100.0,100.0,99.0,95.2,Paraná,Medianeira,13.0
3128,3686,3509700003001,3509700,NaN,FEATURE_02_VALUE_06,False,31.76,FEATURE_05_VALUE_03,4,0.134354,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,São Paulo,Campos do Jordão,25.0


In [36]:
data_merge.to_csv("data/lojas-atuais-uf-mun.csv", index = False)

In [37]:
data_merge.Nome_UF.value_counts()

São Paulo              833
Minas Gerais           279
Paraná                 188
Rio de Janeiro         175
Rio Grande do Sul      175
Bahia                  171
Pernambuco             137
Santa Catarina         119
Ceará                  115
Goiás                   97
Espírito Santo          73
Pará                    68
Maranhão                62
Rio Grande do Norte     60
Mato Grosso             59
Paraíba                 53
Mato Grosso do Sul      53
Alagoas                 43
Piauí                   41
Amazonas                36
Distrito Federal        35
Sergipe                 31
Rondônia                31
Tocantins               18
Amapá                   12
Acre                     8
Roraima                  7
Name: Nome_UF, dtype: int64

In [38]:
data_merge["Nome_Município"].value_counts()

São Paulo         184
Rio de Janeiro     70
Brasília           35
Campinas           33
Recife             33
                 ... 
Antonina            1
Paudalho            1
Barras              1
São Simão           1
Mandaguaçu          1
Name: Nome_Município, Length: 1462, dtype: int64

## Feriados municipais

Cada município tem uma certa quantidade de feriado municipal, devemos utilizar esses dados para [...]

In [30]:
data_feriados_mun_2016 = pd.read_csv("data/feriados-mun-2016.csv")

data_feriados_mun_2016

,data,nome,tipo,descricao,uf,municipio
0,02/11/2016,Finados,MUNICIPAL,Dia de Finados,SC,Criciúma
1,02/11/2016,Finados,MUNICIPAL,Dia de Finados,SP,Monte Mor
2,20/01/2016,São Sebastião,MUNICIPAL,Dia de São Sebastião,ES,Itarana
3,19/08/2016,Nossa Sra da Glória,MUNICIPAL,Nossa Senhora da Glória. Feriado movido para S...,AC,Cruzeiro do Sul
4,30/09/2016,Aniversário da Cidade,MUNICIPAL,Aniversário de Cruzeiro do Sul. Feriado movido...,AC,Cruzeiro do Sul
...,...,...,...,...,...,...
8544,28/07/2016,Feriado Municipal,MUNICIPAL,NaN,TO,Tocantinópolis
8545,15/08/2016,Feriado Municipal,MUNICIPAL,NaN,TO,Tocantinópolis
8546,29/09/2016,Feriado Municipal,MUNICIPAL,NaN,TO,Xambioá
8547,14/11/2016,Feriado Municipal,MUNICIPAL,NaN,TO,Xambioá


In [31]:
data_feriados_mun_2016["datetime"] = pd.to_datetime(data_feriados_mun_2016["data"], format = "%d/%m/%Y")

data_feriados_mun_2016


,data,nome,tipo,descricao,uf,municipio,datetime
0,02/11/2016,Finados,MUNICIPAL,Dia de Finados,SC,Criciúma,2016-11-02
1,02/11/2016,Finados,MUNICIPAL,Dia de Finados,SP,Monte Mor,2016-11-02
2,20/01/2016,São Sebastião,MUNICIPAL,Dia de São Sebastião,ES,Itarana,2016-01-20
3,19/08/2016,Nossa Sra da Glória,MUNICIPAL,Nossa Senhora da Glória. Feriado movido para S...,AC,Cruzeiro do Sul,2016-08-19
4,30/09/2016,Aniversário da Cidade,MUNICIPAL,Aniversário de Cruzeiro do Sul. Feriado movido...,AC,Cruzeiro do Sul,2016-09-30
...,...,...,...,...,...,...,...
8544,28/07/2016,Feriado Municipal,MUNICIPAL,NaN,TO,Tocantinópolis,2016-07-28
8545,15/08/2016,Feriado Municipal,MUNICIPAL,NaN,TO,Tocantinópolis,2016-08-15
8546,29/09/2016,Feriado Municipal,MUNICIPAL,NaN,TO,Xambioá,2016-09-29
8547,14/11/2016,Feriado Municipal,MUNICIPAL,NaN,TO,Xambioá,2016-11-14


In [32]:
data_feriados_mun_2016["mes"] = data_feriados_mun_2016["datetime"].dt.month
data_feriados_mun_2016["ano"] = data_feriados_mun_2016["datetime"].dt.year

data_feriados_mun_2016

,data,nome,tipo,descricao,uf,municipio,datetime,mes,ano
0,02/11/2016,Finados,MUNICIPAL,Dia de Finados,SC,Criciúma,2016-11-02,11,2016
1,02/11/2016,Finados,MUNICIPAL,Dia de Finados,SP,Monte Mor,2016-11-02,11,2016
2,20/01/2016,São Sebastião,MUNICIPAL,Dia de São Sebastião,ES,Itarana,2016-01-20,1,2016
3,19/08/2016,Nossa Sra da Glória,MUNICIPAL,Nossa Senhora da Glória. Feriado movido para S...,AC,Cruzeiro do Sul,2016-08-19,8,2016
4,30/09/2016,Aniversário da Cidade,MUNICIPAL,Aniversário de Cruzeiro do Sul. Feriado movido...,AC,Cruzeiro do Sul,2016-09-30,9,2016
...,...,...,...,...,...,...,...,...,...
8544,28/07/2016,Feriado Municipal,MUNICIPAL,NaN,TO,Tocantinópolis,2016-07-28,7,2016
8545,15/08/2016,Feriado Municipal,MUNICIPAL,NaN,TO,Tocantinópolis,2016-08-15,8,2016
8546,29/09/2016,Feriado Municipal,MUNICIPAL,NaN,TO,Xambioá,2016-09-29,9,2016
8547,14/11/2016,Feriado Municipal,MUNICIPAL,NaN,TO,Xambioá,2016-11-14,11,2016


In [33]:
data_f_municipio = data_feriados_mun_2016.groupby(["municipio"])["mes"].unique()

data_f_municipio

municipio
Abadia dos Dourados           [8]
Abadiânia                     [8]
Abaeté                    [6, 11]
Abaré                   [6, 7, 9]
Abatiá                       [12]
                          ...    
Álvares Machado           [3, 11]
Álvaro de Carvalho           [11]
Áurea                     [8, 11]
Óbidos                     [6, 7]
Óleo                       [4, 8]
Name: mes, Length: 3704, dtype: object

In [34]:
feriados_por_municipio_mes = []

for (index, value) in data_f_municipio.items():
    for mes in value:
        feriados_por_municipio_mes.append([index, mes])

In [35]:
data_feriados_por_municipio_mes = pd.DataFrame(feriados_por_municipio_mes, columns = ["municipio", "mes"])
data_feriados_por_municipio_mes["tem_feriado_municipal"] = True

data_feriados_por_municipio_mes

,municipio,mes,tem_feriado_municipal
0,Abadia dos Dourados,8,True
1,Abadiânia,8,True
2,Abaeté,6,True
3,Abaeté,11,True
4,Abaré,6,True
...,...,...,...
7982,Áurea,11,True
7983,Óbidos,6,True
7984,Óbidos,7,True
7985,Óleo,4,True


In [ ]:
data_feriados_por_municipio_mes.to_csv("data/feriados-municipais")

## População estimada dos municípios

In [142]:
data_pop_estim = pd.read_excel("ibge/tabela6579.xlsx", skiprows = [0, 1, 2, 3], names = ["cod_municipio", "Nome_Município (UF)", "popEstim"], skipfooter = 1)

data_pop_estim

,cod_municipio,Nome_Município (UF),popEstim
0,1100023,Ariquemes (RO),105896
1,1100031,Cabixi (RO),6289
2,1100049,Cacoal (RO),87877
3,1100056,Cerejeiras (RO),17959
4,1100064,Colorado do Oeste (RO),18639
...,...,...,...
5564,5222005,Vianópolis (GO),13567
5565,5222054,Vicentinópolis (GO),8286
5566,5222203,Vila Boa (GO),5615
5567,5222302,Vila Propício (GO),5635


## Feriados nacionais de 2016 a 2017

Vamos organizar os feriados nacionais de 2016 a 2017 por mês.

In [10]:
data_feriados_nacionais = pd.read_csv("data/feriados-nacionais-2016a2017.csv")

data_feriados_nacionais

,data,nome,tipo,descricao,uf,municipio
0,01/01/2016,Ano Novo,NACIONAL,O Ano-Novo ou Réveillon é um evento que aconte...,NaN,
1,25/03/2016,Sexta-Feira Santa,NACIONAL,"Também chamada de ""Sexta Feira da Paixão"" é a ...",NaN,
2,21/04/2016,Dia de Tiradentes,NACIONAL,"Joaquim José da Silva Xavier, o Tiradentes foi...",NaN,
3,01/05/2016,Dia do Trabalho,NACIONAL,O Dia do Trabalhador ou Dia Internacional dos ...,NaN,
4,07/09/2016,Independência do Brasil,NACIONAL,"O Dia da Indepência do Brasil, oficialmente a ...",NaN,
5,12/10/2016,Nossa Senhora Aparecida,NACIONAL,Nossa Senhora da Conceição Aparecida é a padro...,NaN,
6,02/11/2016,Dia de Finados,NACIONAL,"O Dia de Finados ou Dia dos Fiéis Defuntos, (c...",NaN,
7,15/11/2016,Proclamação da República,NACIONAL,A Proclamação da República foi um levante polí...,NaN,
8,25/12/2016,Natal,NACIONAL,O Natal é comemorado anualmente em 25 de Dezem...,NaN,
9,01/01/2017,Ano Novo,NACIONAL,O Ano-Novo ou Réveillon é um evento que aconte...,NaN,


In [11]:
data_feriados_nacionais["datetime"] = pd.to_datetime(data_feriados_nacionais["data"], format = "%d/%m/%Y")

data_feriados_nacionais


,data,nome,tipo,descricao,uf,municipio,datetime
0,01/01/2016,Ano Novo,NACIONAL,O Ano-Novo ou Réveillon é um evento que aconte...,NaN,,2016-01-01
1,25/03/2016,Sexta-Feira Santa,NACIONAL,"Também chamada de ""Sexta Feira da Paixão"" é a ...",NaN,,2016-03-25
2,21/04/2016,Dia de Tiradentes,NACIONAL,"Joaquim José da Silva Xavier, o Tiradentes foi...",NaN,,2016-04-21
3,01/05/2016,Dia do Trabalho,NACIONAL,O Dia do Trabalhador ou Dia Internacional dos ...,NaN,,2016-05-01
4,07/09/2016,Independência do Brasil,NACIONAL,"O Dia da Indepência do Brasil, oficialmente a ...",NaN,,2016-09-07
5,12/10/2016,Nossa Senhora Aparecida,NACIONAL,Nossa Senhora da Conceição Aparecida é a padro...,NaN,,2016-10-12
6,02/11/2016,Dia de Finados,NACIONAL,"O Dia de Finados ou Dia dos Fiéis Defuntos, (c...",NaN,,2016-11-02
7,15/11/2016,Proclamação da República,NACIONAL,A Proclamação da República foi um levante polí...,NaN,,2016-11-15
8,25/12/2016,Natal,NACIONAL,O Natal é comemorado anualmente em 25 de Dezem...,NaN,,2016-12-25
9,01/01/2017,Ano Novo,NACIONAL,O Ano-Novo ou Réveillon é um evento que aconte...,NaN,,2017-01-01


In [12]:
data_feriados_nacionais["mes"] = data_feriados_nacionais["datetime"].dt.month
data_feriados_nacionais["ano"] = data_feriados_nacionais["datetime"].dt.year

data_feriados_nacionais

,data,nome,tipo,descricao,uf,municipio,datetime,mes,ano
0,01/01/2016,Ano Novo,NACIONAL,O Ano-Novo ou Réveillon é um evento que aconte...,NaN,,2016-01-01,1,2016
1,25/03/2016,Sexta-Feira Santa,NACIONAL,"Também chamada de ""Sexta Feira da Paixão"" é a ...",NaN,,2016-03-25,3,2016
2,21/04/2016,Dia de Tiradentes,NACIONAL,"Joaquim José da Silva Xavier, o Tiradentes foi...",NaN,,2016-04-21,4,2016
3,01/05/2016,Dia do Trabalho,NACIONAL,O Dia do Trabalhador ou Dia Internacional dos ...,NaN,,2016-05-01,5,2016
4,07/09/2016,Independência do Brasil,NACIONAL,"O Dia da Indepência do Brasil, oficialmente a ...",NaN,,2016-09-07,9,2016
5,12/10/2016,Nossa Senhora Aparecida,NACIONAL,Nossa Senhora da Conceição Aparecida é a padro...,NaN,,2016-10-12,10,2016
6,02/11/2016,Dia de Finados,NACIONAL,"O Dia de Finados ou Dia dos Fiéis Defuntos, (c...",NaN,,2016-11-02,11,2016
7,15/11/2016,Proclamação da República,NACIONAL,A Proclamação da República foi um levante polí...,NaN,,2016-11-15,11,2016
8,25/12/2016,Natal,NACIONAL,O Natal é comemorado anualmente em 25 de Dezem...,NaN,,2016-12-25,12,2016
9,01/01/2017,Ano Novo,NACIONAL,O Ano-Novo ou Réveillon é um evento que aconte...,NaN,,2017-01-01,1,2017


In [13]:
data_feriados_nacionais = data_feriados_nacionais[["mes", "ano"]]

data_feriados_nacionais

,mes,ano
0,1,2016
1,3,2016
2,4,2016
3,5,2016
4,9,2016
5,10,2016
6,11,2016
7,11,2016
8,12,2016
9,1,2017


In [19]:
contagem_feriados = data_feriados_nacionais.groupby(["ano"])["mes"].value_counts().to_frame("contagem")

contagem_feriados

contagem
ano  mes          
2016 11          2
     1           1
     3           1
     4           1
     5           1
     9           1
     10          1
     12          1
2017 4           2
     11          2
     1           1
     5           1
     9           1
     10          1
     12          1

In [20]:
contagem_feriados.reset_index()

,ano,mes,contagem
0,2016,11,2
1,2016,1,1
2,2016,3,1
3,2016,4,1
4,2016,5,1
5,2016,9,1
6,2016,10,1
7,2016,12,1
8,2017,4,2
9,2017,11,2


In [21]:
contagem_feriados = contagem_feriados.reset_index()

contagem_feriados.to_csv("data/meses-com-feriado-nacional.csv", index = False)